<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Latest_Deepfake_Detection_and_Manipulated_Media_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Deepfake Detection and Manipulated Media Analysis using Multiagent System and Compound AI Approach**

In [1]:
%pip install torch opencv-python librosa numpy face-recognition
%pip install vllm transformers mediapipe scipy pillow tqdm pydantic moviepy langchain_community langgraph dtw-python
%pip install ipywidgets nest_asyncio librosa groq

In [2]:
import os
import torch
import cv2
import numpy as np
import librosa
import asyncio
import json
import re
import gc
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from typing import Dict, List, Any, Tuple, Optional

import mediapipe as mp
from pydantic import BaseModel, Field

# Import for LLMs and chain operations
from langchain_community.llms import VLLM, VLLMOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langgraph.graph import StateGraph, END

from moviepy.editor import VideoFileClip

import nest_asyncio
nest_asyncio.apply()

import ipywidgets as widgets
from IPython.display import display, clear_output

# Additional imports for image and video quality metrics
from skimage.metrics import structural_similarity as ssim

# For lip-sync DTW computation
from dtw import dtw

# For face recognition
import face_recognition

  if event.key is 'enter':



Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
# Set CUDA environment variables and clear GPU memory
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
torch.cuda.empty_cache()

In [4]:
# Common parameters for model initialization
COMMON_PARAMS = {
    "task": "generate",
    "max_model_len": 4096,
    "dtype": "half",
    "gpu_memory_utilization": 0.85,
    "cpu_offload_gb": 8,
    "enforce_eager": True,
    "trust_remote_code": True
}

def init_vllm_model(name: str, model_id: str, **overrides) -> Dict[str, Any]:
    params = {**COMMON_PARAMS, **overrides}
    print(f"Initializing model '{name}' with id '{model_id}' with params: {params}")
    return {"name": name, "model_id": model_id, "params": params}

def init_groq_model(name: str, model_id: str) -> Dict[str, Any]:
    api_key = os.environ.get("GROQ_API_KEY", "your_groq_api_key")
    print(f"Loading model '{name}' with id '{model_id}' using API key.")
    return {"name": name, "model_id": model_id, "api_key": api_key}

# Initialize models for deepfake analysis
print("Initializing models...")
models = {
    "video": [
        init_vllm_model("llava_next_video", "llava-hf/LLaVA-NeXT-Video-7B-hf", tensor_parallel_size=2, max_tokens=1024),
        init_vllm_model("videomae", "MCG-NJU/videomae-large-static", tensor_parallel_size=2),
        init_vllm_model("mdapt", "google/mdapt-large", tensor_parallel_size=2)
    ],
    "audio": [
        init_vllm_model("wav2vec2", "facebook/wav2vec2-large-robust-ft-swbd-300h", tensor_parallel_size=1),
        init_vllm_model("whisper", "openai/whisper-large-v3", tensor_parallel_size=2),
        init_groq_model("groq_audio_model", "whisper-large-v3-turbo")
    ],
    "image": [
        init_vllm_model("llava_image", "llava-hf/llava-onevision-qwen2-7b-ov-hf", tensor_parallel_size=2),
        init_vllm_model("clip", "openai/clip-vit-large-patch14", tensor_parallel_size=1),
        init_groq_model("groq_vision_model", "llama-3.2-90b-vision-preview")
    ],
    "text": [
        {"name": "vllm_chain", "instance": VLLM(
            model="stabilityai/stablelm-tuned-alpha-7b",
            tensor_parallel_size=2,
            trust_remote_code=True,
            max_new_tokens=512,
            top_k=40,
            top_p=0.95,
            temperature=0.8,
            vllm_kwargs={}
        )},
        init_groq_model("groq_text_model", "llama-3.3-70b-versatile"),
        init_groq_model("groq_llama_scout", "meta-llama/llama-4-scout-17b-16e-instruct"),
        init_groq_model("groq_llama_maverick", "meta-llama/llama-4-maverick-17b-128e-instruct")
    ]
}

Initializing models...
Initializing model 'llava_next_video' with id 'llava-hf/LLaVA-NeXT-Video-7B-hf' with params: {'task': 'generate', 'max_model_len': 4096, 'dtype': 'half', 'gpu_memory_utilization': 0.85, 'cpu_offload_gb': 8, 'enforce_eager': True, 'trust_remote_code': True, 'tensor_parallel_size': 2, 'max_tokens': 1024}
Initializing model 'videomae' with id 'MCG-NJU/videomae-large-static' with params: {'task': 'generate', 'max_model_len': 4096, 'dtype': 'half', 'gpu_memory_utilization': 0.85, 'cpu_offload_gb': 8, 'enforce_eager': True, 'trust_remote_code': True, 'tensor_parallel_size': 2}
Initializing model 'mdapt' with id 'google/mdapt-large' with params: {'task': 'generate', 'max_model_len': 4096, 'dtype': 'half', 'gpu_memory_utilization': 0.85, 'cpu_offload_gb': 8, 'enforce_eager': True, 'trust_remote_code': True, 'tensor_parallel_size': 2}
Initializing model 'wav2vec2' with id 'facebook/wav2vec2-large-robust-ft-swbd-300h' with params: {'task': 'generate', 'max_model_len': 4096

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

INFO 04-08 14:10:35 [config.py:2700] Downcasting torch.float32 to torch.float16.
INFO 04-08 14:10:49 [config.py:600] This model supports multiple tasks: {'classify', 'generate', 'reward', 'embed', 'score'}. Defaulting to 'generate'.
WARNING 04-08 14:10:49 [arg_utils.py:1708] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 04-08 14:10:49 [config.py:1600] Defaulting to use ray for distributed inference
WARNING 04-08 14:10:49 [config.py:1262] Possibly too large swap space. 8.00 GiB out of the 12.67 GiB total CPU memory is allocated for the swap space.
INFO 04-08 14:10:49 [llm_engine.py:242] Initializing a V0 LLM engine (v0.8.3) with config: model='stabilityai/stablelm-tuned-alpha-7b', speculative_config=None, tokenizer='stabilityai/stablelm-tuned-alpha-7b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

2025-04-08 14:10:54,291	INFO worker.py:1841 -- Started a local Ray instance.


INFO 04-08 14:10:55 [ray_utils.py:335] No current placement group found. Creating a new placement group.
WARNING 04-08 14:10:55 [ray_utils.py:342] The number of required GPUs exceeds the total number of available GPUs in the placement group.
INFO 04-08 14:11:05 [ray_utils.py:233] Waiting for creating a placement group of specs for 10 seconds. specs=[{'node:172.28.0.12': 0.001, 'GPU': 1.0}, {'GPU': 1.0}]. Check `ray status` and `ray list nodes` to see if you have enough resources, and make sure the IP addresses used by ray cluster are the same as VLLM_HOST_IP environment variable specified in each node if you are running on a multi-node.
INFO 04-08 14:11:25 [ray_utils.py:233] Waiting for creating a placement group of specs for 30 seconds. specs=[{'node:172.28.0.12': 0.001, 'GPU': 1.0}, {'GPU': 1.0}]. Check `ray status` and `ray list nodes` to see if you have enough resources, and make sure the IP addresses used by ray cluster are the same as VLLM_HOST_IP environment variable specified i

KeyboardInterrupt: 